In [3]:

!pip install chess-board
!pip install chess


In [4]:
"""
Artificial Intelligence – Assignment 3
Submission Deadline - 05/09/2025
--------------------------------------

Topic: Chess AI with Minimax (Alpha-Beta) and Evaluation Function

This version implements the `evaluate` method in the State class.
"""

import chess
from chessboard import display
import time

class State:
    def __init__(self, board=None, player=True):
        if board is None:
            self.board = chess.Board()
        else:
            self.board = board
        self.player = player  # True = White's turn, False = Black's turn

    def goalTest(self):
        # Check if the game is over
        if self.board.is_checkmate():
            return not self.player  # The opponent just made a winning move
        return None

    def isTerminal(self):
        return self.board.is_game_over()

    def moveGen(self):
        # Generate next states
        children = []
        for move in self.board.legal_moves:
            new_board = self.board.copy()
            new_board.push(move)
            children.append(State(new_board, not self.player))
        return children

    def __str__(self):
        return str(self.board)

    def __eq__(self, other):
        return self.board.fen() == other.board.fen() and self.player == other.player

    def __hash__(self):
        return hash((self.board.fen(), self.player))

    def evaluate(self):
        """
        Evaluation function for chess positions.

        Returns a numeric score: positive -> good for White, negative -> good for Black.
        This implementation combines:
         - material (major weight)
         - simple piece-square tables for pawns/knights (small improvement)
         - center control (d4,e4,d5,e5)
         - mobility (difference in legal moves)
         - king safety (number of attackers on the king squares)

        Values and weights are intentionally simple and tuned for clarity/assignment use.
        """

        # Step 1: Handle finished games
        if self.board.is_checkmate():
            # If it's checkmate and it's currently player's turn, that player was mated.
            # Return large negative if White to move and mated, large positive if Black to move and mated.
            return -10000 if self.player else 10000
        if self.board.is_stalemate() or self.board.is_insufficient_material() or self.board.can_claim_draw():
            return 0

        # Piece base values
        piece_values = {
            chess.PAWN: 100,
            chess.KNIGHT: 320,
            chess.BISHOP: 330,
            chess.ROOK: 500,
            chess.QUEEN: 900,
            chess.KING: 20000,
        }

        score = 0

        # Simple piece-square tables (small bonuses). Values in centipawns.
        # These are simplistic and symmetric for brevity.
        pawn_table = [
            0, 0, 0, 0, 0, 0, 0, 0,
            5, 10, 10, -20, -20, 10, 10, 5,
            5, -5, -10, 0, 0, -10, -5, 5,
            0, 0, 0, 20, 20, 0, 0, 0,
            5, 5, 10, 25, 25, 10, 5, 5,
            10, 10, 20, 30, 30, 20, 10, 10,
            50, 50, 50, 50, 50, 50, 50, 50,
            0, 0, 0, 0, 0, 0, 0, 0,
        ]

        knight_table = [
            -50, -40, -30, -30, -30, -30, -40, -50,
            -40, -20, 0, 5, 5, 0, -20, -40,
            -30, 5, 10, 15, 15, 10, 5, -30,
            -30, 0, 15, 20, 20, 15, 0, -30,
            -30, 5, 15, 20, 20, 15, 5, -30,
            -30, 0, 10, 15, 15, 10, 0, -30,
            -40, -20, 0, 0, 0, 0, -20, -40,
            -50, -40, -30, -30, -30, -30, -40, -50,
        ]

        # (a) MATERIAL + piece-square tables
        for sq, piece in self.board.piece_map().items():
            value = piece_values.get(piece.piece_type, 0)
            # piece-square table adjustment (centipawns -> same units as value)
            pst = 0
            if piece.piece_type == chess.PAWN:
                # python-chess uses 0=a1..63=h8; our tables are white-perspective left-to-right, top-to-bottom
                pst = pawn_table[sq] / 100.0
            elif piece.piece_type == chess.KNIGHT:
                pst = knight_table[sq] / 100.0

            signed_value = value / 100.0 + pst  # convert centipawn-like base to pawn units
            if piece.color == chess.WHITE:
                score += signed_value
            else:
                score -= signed_value

        # (b) CENTER CONTROL
        center_squares = [chess.D4, chess.E4, chess.D5, chess.E5]
        for sq in center_squares:
            piece = self.board.piece_at(sq)
            if piece:
                # small bonus for occupying center
                if piece.color == chess.WHITE:
                    score += 0.25
                else:
                    score -= 0.25

        # (c) MOBILITY
        # Count legal moves for both sides (without changing the real board turn)
        b = self.board.copy()
        b.turn = chess.WHITE
        white_moves = len(list(b.legal_moves))
        b.turn = chess.BLACK
        black_moves = len(list(b.legal_moves))
        score += 0.05 * (white_moves - black_moves)

        # (d) KING SAFETY
        # Penalize if king is attacked by opponent pieces (simple count)
        white_king_sq = self.board.king(chess.WHITE)
        black_king_sq = self.board.king(chess.BLACK)
        if white_king_sq is not None:
            attackers_on_white = len(self.board.attackers(chess.BLACK, white_king_sq))
            score -= 0.5 * attackers_on_white
        if black_king_sq is not None:
            attackers_on_black = len(self.board.attackers(chess.WHITE, black_king_sq))
            score += 0.5 * attackers_on_black

        # Extra small bonus for bishop pair
        white_bishops = len(self.board.pieces(chess.BISHOP, chess.WHITE))
        black_bishops = len(self.board.pieces(chess.BISHOP, chess.BLACK))
        if white_bishops >= 2:
            score += 0.25
        if black_bishops >= 2:
            score -= 0.25

        # Return final score (positive -> White advantage)
        return score


def minimax(state, depth, alpha, beta, maximizingPlayer, maxDepth):
    if state.isTerminal() or depth == maxDepth:
        return state.evaluate(), None

    best_move = None

    if maximizingPlayer:  # MAX node (White)
        maxEval = float('-inf')
        for child in state.moveGen():
            eval_score, _ = minimax(child, depth + 1, alpha, beta, False, maxDepth)

            if eval_score > maxEval:
                maxEval = eval_score
                best_move = child.board.peek()  # Last move made

            alpha = max(alpha, eval_score)
            if alpha >= beta:
                break  # Alpha-beta pruning

        return maxEval, best_move

    else:  # MIN node (Black)
        minEval = float('inf')
        for child in state.moveGen():
            eval_score, _ = minimax(child, depth + 1, alpha, beta, True, maxDepth)

            if eval_score < minEval:
                minEval = eval_score
                best_move = child.board.peek()

            beta = min(beta, eval_score)
            if alpha >= beta:
                break

        return minEval, best_move


def play_game():
    current_state = State(player=True)  # White starts
    maxDepth = 3  # Try experimenting with the Search depth for more inteligent ai
    game_board = display.start()  # Initialize the GUI

    print("Artificial Intelligence – Assignment 3")
    print("Simple Chess AI")
    print("You are playing as White (enter moves in UCI format, e.g., e2e4)")

    while not current_state.isTerminal():
        # Update the display
        display.update(current_state.board.fen(), game_board)

        # Check for quit event
        if display.check_for_quit():
            break

        if current_state.player:  # Human move (White)
            try:
                move_uci = input("Enter your move (e.g., e2e4, g1f3, a7a8q) or 'quit': ")

                if move_uci.lower() == 'quit':
                    break

                move = chess.Move.from_uci(move_uci)

                if move in current_state.board.legal_moves:
                    new_board = current_state.board.copy()
                    new_board.push(move)
                    current_state = State(new_board, False)
                else:
                    print("Invalid move! Try again.")
                    continue
            except ValueError:
                print("Invalid input format! Use UCI format like 'e2e4'.")
                continue
        else:  # AI move (Black)
            print("AI is thinking...")
            start_time = time.time()
            eval_score, best_move = minimax(current_state, 0, float('-inf'), float('inf'), False, maxDepth)
            end_time = time.time()

            print(f"AI thought for {end_time - start_time:.2f} seconds")

            if best_move:
                new_board = current_state.board.copy()
                new_board.push(best_move)
                current_state = State(new_board, True)
                print(f"AI plays: {best_move.uci()}")
            else:
                # Fallback
                legal_moves = list(current_state.board.legal_moves)
                if legal_moves:
                    move = legal_moves[0]
                    new_board = current_state.board.copy()
                    new_board.push(move)
                    current_state = State(new_board, True)
                    print(f"AI plays (fallback): {move.uci()}")
                else:
                    break

    # Game over
    print("\nGame over!")
    display.update(current_state.board.fen(), game_board)

    if current_state.board.is_checkmate():
        print("Checkmate! " + ("White" if not current_state.player else "Black") + " wins!")
    elif current_state.board.is_stalemate():
        print("Stalemate! It's a draw.")
    elif current_state.board.is_insufficient_material():
        print("Insufficient material! It's a draw.")
    elif current_state.board.can_claim_draw():
        print("Draw by repetition or 50-move rule!")

    # Keep the window open for a moment
    time.sleep(3)
    display.terminate()


if __name__ == "__main__":
    play_game()

Artificial Intelligence – Assignment 3
Simple Chess AI
You are playing as White (enter moves in UCI format, e.g., e2e4)
Enter your move (e.g., e2e4, g1f3, a7a8q) or 'quit': e2e4
AI is thinking...
AI thought for 0.76 seconds
AI plays: g8f6
Enter your move (e.g., e2e4, g1f3, a7a8q) or 'quit': g1f3
AI is thinking...
AI thought for 0.76 seconds
AI plays: b8c6
Enter your move (e.g., e2e4, g1f3, a7a8q) or 'quit': quit

Game over!


SystemExit: 

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
